<h1> Define constants and variables </h1>

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from docplex.mp.model import Model

class CustomCCPPSolver:
    def __init__(self, params):
        self.solution = None
        self.params = params
        
    def solve(self):
        (K, V, V_count, theta, L, l_i, V_loc_x, V_loc_y, E, d, TV, SV, G) = self.params
        radius = CapControllerPlacement(V, d, l_i, L, K)
        if(radius > 0):
            solution = SP(radius, V, d, l_i, L)
            self.solution = solution
        else:
            print("No solution for given constraints")
        
    def plotSolution(self):
        if self.solution:
            (K, V, V_count, theta, L, l_i, V_loc_x, V_loc_y, E, d, TV, SV, G) = self.params
            G_nodes = list(G.nodes)
            num_of_controllers, y_j, x_ij = self.solution
            colors = {}
            fig, ax = plt.subplots(figsize=(15,15))
            active_arcs = [a for a in x_ij if x_ij[a].solution_value>0.9]

            for i,j in active_arcs:
                path = nx.shortest_path(G, G_nodes[i], G_nodes[j], 'weight')
                if not path[-1] in colors:
                    colors[path[-1]] = np.random.rand(3,)

                for k in range(0, len(path)-1):
                    vi = G_nodes.index(path[k])
                    vj = G_nodes.index(path[k+1])
                    plt.plot([V_loc_x[vi], V_loc_x[vj]], [V_loc_y[vi], V_loc_y[vj]],c = colors[path[-1]], linewidth=4)
                    if k == len(path)-2:
                        plt.plot(V_loc_x[vj], V_loc_y[vj], c=colors[path[-1]], marker='s', markersize=15)

                if len(path)-1 == 0:
                    vi = G_nodes.index(path[0])
                    plt.plot(V_loc_x[vi], V_loc_y[vi], c=colors[path[-1]], marker='s', markersize=15)
            plt.scatter(V_loc_x[:], V_loc_y[:])
            for i in V:
                plt.annotate('   $%s$' % (G_nodes[i]), (V_loc_x[i], V_loc_y[i]))
                plt.axis('equal')
        else:
            print('No solution to plot')
        

In [2]:
def i_indexesM(ij_pairs):
    return list(set([x[0] for x in ij_pairs]))


def j_indexes(xPairs, i):
    return map(lambda x: x[1], list(filter(lambda x: x[0] == i, xPairs)))


def j_indexesM(ij_pairs):
    return list(set([x[1] for x in ij_pairs]))


def i_indexes(xPairs, j):
    return map(lambda x: x[0], list(filter(lambda x: x[1] == j, xPairs)))

<h2> Subproblem SP - minimizing the number of factories for given radius</h2>

In [3]:
def SP(eps, nodes, d, h, Q):
    mdl = Model('CCPP',checker=False)

    # zmienne lokalizacji
    y_j = mdl.binary_var_dict(nodes, name="y_j")

    # pary (i,j) takie, ze d_ij <= epsilon
    xPairs = []
    for i in range(np.size(nodes)):
        for j in range(np.size(nodes)):
            if d[i, j] <= eps:
                xPairs.append((i, j))
    # zmienne przypisania
    x_ij = mdl.binary_var_dict(xPairs, name="x_ij")

    
    # minimalizuj liczbe fabryk
    mdl.minimize(mdl.sum(y_j[i] for i in nodes))

    
    # kazdy klient do jednej fabryki
    mdl.add_constraints(mdl.sum(x_ij[i, j] for j in j_indexes(xPairs, i)) == 1 for i in i_indexesM(xPairs))
   
    # dla kazdej fabryki, suma zapotrzebowania klientow mniejsza lub rowna pojemnosci fabryki
    mdl.add_constraints(mdl.sum(h[i] * x_ij[i, j] for i in i_indexes(xPairs, j)) <= Q[j] 
                        for j in j_indexesM(xPairs))
    
    # klient moze byc przypisany do istniejacej fabryki
    mdl.add_constraints(x_ij[i, j] <= y_j[j] for (i, j) in xPairs)

    
    mdl.parameters.timelimit = 30
    solution = mdl.solve()
    if solution:
        number_of_controllers = int(sum([y_j[a].solution_value for a in nodes]))
        return number_of_controllers, y_j, x_ij
    return -1

<h2> Linear relaxation of x_ij variable in subproblem SP (binary variable changed to continuous variable) </h2>

In [4]:
def SP_LR(eps, nodes, d, h, Q):
    from docplex.mp.model import Model
    mdl = Model('CCPP')

    # zmienne lokalizacji
    y_j = mdl.binary_var_dict(nodes, name="y_j")

    # pary (i,j) takie, ze d_ij <= epsilon
    xPairs = []
    for i in range(np.size(nodes)):
        for j in range(np.size(nodes)):
            if d[i, j] <= eps:
                xPairs.append((i, j))
    # zmienne przypisania
    x_ij = mdl.continuous_var_dict(xPairs, name="x_ij")

    
    # minimalizuj liczbe fabryk
    mdl.minimize(mdl.sum(y_j[i] for i in nodes))

    
    # kazdy klient do jednej fabryki
    mdl.add_constraints(mdl.sum(x_ij[i, j] for j in j_indexes(xPairs, i)) == 1 for i in i_indexesM(xPairs))
   
    # dla kazdej fabryki, suma zapotrzebowania klientow mniejsza lub rowna pojemnosci fabryki
    mdl.add_constraints(mdl.sum(h[i] * x_ij[i, j] for i in i_indexes(xPairs, j)) <= Q[j] 
                        for j in j_indexesM(xPairs))
    
    # klient moze byc przypisany do istniejacej fabryki
    mdl.add_constraints(x_ij[i, j] <= y_j[j] for (i, j) in xPairs)
    
    # 0 <= x_ij <= 1.0 
    mdl.add_constraints(x_ij[i, j] <= 1.0 for (i, j) in xPairs)
    mdl.add_constraints(x_ij[i, j] >= 0.0 for (i, j) in xPairs)

    
    mdl.parameters.timelimit = 30
    solution = mdl.solve()
    if solution:
        number_of_controllers = int(sum([y_j[a].solution_value for a in nodes]))
        return number_of_controllers, y_j, x_ij
    return -1

<h2>The algorithm </h2>

In [5]:
def CapControllerPlacement(nodes, d, l_i, L, K):                   
    disArray = list(set(np.array(d).flatten().tolist()))
    disArray.sort()
    lower = 0
    upper = len(disArray) - 1
    while lower < upper:
        mid = (lower + upper)//2
        r = disArray[mid]
        num_of_controllers,_,_ = SP_LR(r, nodes, d, l_i, L)
        if num_of_controllers > K:
            lower = mid + 1
        else:
            upper = mid
    index = lower
    num_of_controllers,_,_ = SP(disArray[mid], nodes, d, l_i, L)
    
    while num_of_controllers > K:
        index += 1
        if(index >= len(disArray)):
            return -1;
        num_of_controllers,_,_ = SP(disArray[index], nodes, d, l_i, L)
    return disArray[index]